In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
os.environ['TEJAPI_KEY'] = "ab0aChJaHZYJXVpENhmMQylbwTWO4a"
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw"

In [2]:
from zipline.finance import slippage, commission
from zipline.api import order_target, record, symbol, set_benchmark, set_commission, set_slippage, attach_pipeline, pipeline_output
import matplotlib.pyplot as plt
from zipline import run_algorithm
from zipline.sources.TEJ_Api_Data import get_universe
from zipline.data import bundles
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.data import TWEquityPricing
from zipline.pipeline.filters import StaticAssets
from zipline.pipeline.mixins import SingleInputMixin
from zipline.pipeline.factors import RSI
from zipline.utils.math_utils import nanmean
from numpy import abs, clip, diff, inf
from numexpr import evaluate
start = '2021-01-01'
end = '2023-12-31'
StockList = ['2330','2454','2455','2458','3014','3034','3529','4952','4961','5234','6183','6531','6756','6799','8016','8081','8261']
tickers = ' '.join(StockList)
os.environ['ticker'] = tickers + ' IX0001'
os.environ['mdate'] = start + ' ' + end

In [10]:
# calendar------------------------------------------
calendar_name = 'TEJ_XTAI'  
# bundle_name---------------------------------------
bundle_name = 'tquant'

# 确保 zipline 数据包已导入
!zipline ingest -b tquant

[2024-06-13 02:17:49.383379] INFO: zipline.data.bundles.core: Ingesting tquant.
Merging daily equity files:  [####################################]    
Currently used TEJ API key call quota 62/1000 (6.2%)
Currently used TEJ API key data quota 244077/10000000 (2.44%)
[2024-06-13 02:17:52.991502] INFO: zipline.data.bundles.core: Ingest tquant successfully.


In [11]:
class RSI2(SingleInputMixin, CustomFactor):
    window_length = 15
    inputs = (TWEquityPricing.close,)
    window_safe = True

    def compute(self, today, assets, out, closes):
        diffs = diff(closes[:-1][:], axis=0)
        ups = nanmean(clip(diffs, 0, inf), axis=0)
        downs = abs(nanmean(clip(diffs, -inf, 0), axis=0))
        return evaluate(
            "100 - (100 / (1 + (ups / downs)))",
            local_dict={'ups': ups, 'downs': downs},
            global_dict={},
            out=out,
        )


In [12]:
def make_pipeline(short, long):
    rsi_long = RSI(inputs=[TWEquityPricing.close], window_length=long)
    rsi_short = RSI(inputs=[TWEquityPricing.close], window_length=short)
    pre_rsi_long = RSI2(inputs=[TWEquityPricing.close], window_length=long+1)
    pre_rsi_short = RSI2(inputs=[TWEquityPricing.close], window_length=short+1)

    return Pipeline(columns={
        'curr_price': TWEquityPricing.close.latest,
        'RSI_short': rsi_short,
        'RSI_long': rsi_long,
        'pre_RSI_short': pre_rsi_short,
        'pre_RSI_long': pre_rsi_long
    })

In [13]:
def initialize(context):
    set_slippage(slippage.VolumeShareSlippage())
    set_commission(commission.PerShare(cost=0.001425 + 0.003 / 2))
    attach_pipeline(make_pipeline(6, 12), 'mystrats')
    set_benchmark(symbol('IX0001'))
    context.sym = symbol('2330')
    context.i = 0
    context.invested = False


In [14]:
def handle_data(context, data):
    out_dir = pipeline_output('mystrats')
    trailing_window = data.history(context.sym, 'price', 35, '1d')
    if trailing_window.isnull().values.any():
        return

    short_ema = talib.EMA(trailing_window.values, timeperiod=12)
    long_ema = talib.EMA(trailing_window.values, timeperiod=26)
    dif = short_ema - long_ema
    MACD = talib.EMA(dif, timeperiod=9)
    bar = dif - MACD

    for i in out_dir.index:
        sym = i.symbol
        RSI_short = out_dir.loc[i, 'RSI_short']
        RSI_long = out_dir.loc[i, "RSI_long"]
        pre_RSI_short = out_dir.loc[i, 'pre_RSI_short']
        pre_RSI_long = out_dir.loc[i, "pre_RSI_long"]
        curr_price = out_dir.loc[i, 'curr_price']

        cash_position = context.portfolio.cash
        stock_position = context.portfolio.positions[i].amount

        buy, sell = False, False

        print(f"Symbol: {sym}, RSI_short: {RSI_short}, RSI_long: {RSI_long}, dif[-2]: {dif[-2]}, MACD[-2]: {MACD[-2]}, dif[-1]: {dif[-1]}, MACD[-1]: {MACD[-1]}")

        if stock_position >= 0:
            if (RSI_short <= 30) and (dif[-2] < MACD[-2]) and (dif[-1] > MACD[-1]):
                order_target(i, 1000)
                buy = True
                print(f"Buying {sym} at {curr_price}")

        elif stock_position > 0:
            if (RSI_short >= 70) and (dif[-2] > MACD[-2]) and (dif[-1] < MACD[-1]):
                order_target(i, 0)
                sell = True
                print(f"Selling {sym} at {curr_price}")

        record(
            TSMC=data.current(symbol('2330'), 'close'),
            dif=dif[-1],
            MACD=MACD[-1],
            bar=bar[-1],
            buy=buy,
            sell=sell,
            RSI_short_2330=RSI_short,
            RSI_long_2330=RSI_long
        )


In [15]:
def analyze(context=None, results=None):
    import matplotlib.pyplot as plt
    import logbook
    logbook.StderrHandler().push_application()
    log = logbook.Logger('Algorithm')

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(18, 15), sharex=True)

    results.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('Portfolio value (TWD)')

    if 'MACD' in results and 'bar' in results:
        ax2.plot(results.index, results['MACD'], label='MACD', color='blue')
        colors = ["red" if bar > 0 else "green" for bar in results['bar']]
        ax2.bar(results.index, results['bar'], color=colors, alpha=0.5, width=0.4, label='MACD Bar')

        ax2.plot(
            results.index[results["buy"] == True],
            results.loc[results["buy"] == True, 'MACD'],
            '^',
            markersize=10,
            color='m',
            label='Buy Signal'
        )
        ax2.plot(
            results.index[results["sell"] == True],
            results.loc[results["sell"] == True, 'MACD'],
            'v',
            markersize=10,
            color='k',
            label='Sell Signal'
        )

        ax2.set_ylabel('MACD')
        ax2.legend(loc='upper left')

    else:
        msg = 'MACD and MACD Bar data not captured using record().'
        ax2.annotate(msg, xy=(0.1, 0.5))
        log.info(msg)

    if 'RSI_short_2330' in results:
        ax3.plot(results.index, results['RSI_short_2330'], label='RSI Short', color='orange')
        ax3.plot(results.index, results['RSI_long_2330'], label='RSI Long', color='purple')
        ax3.set_ylabel('RSI')
        ax3.legend(loc='upper right')

    else:
        msg = 'RSI data not captured using record().'
        ax3.annotate(msg, xy=(0.1, 0.5))
        log.info(msg)

    plt.tight_layout()
    plt.show()


In [9]:
start_d = pd.Timestamp('2021-01-22', tz='utc')
end_date = pd.Timestamp('2023-12-31', tz='utc')
results = run_algorithm(
    start=start_d,
    end=end_date,
    initialize=initialize,
    bundle='tquant',
    analyze=analyze,
    capital_base=5e6,
    handle_data=handle_data
)

NameError: name 'start_date' is not defined

In [ ]:
results.returns.cumsum().plot()
plt.show()


In [ ]:
import pyfolio as pf
from pyfolio.utils import extract_rets_pos_txn_from_zipline

returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(results)
benchmark_rets = results.benchmark_return

# Creating a Full Tear Sheet
pf.create_full_tear_sheet(returns, positions = positions, transactions = transactions,
                          benchmark_rets = benchmark_rets,
                          round_trips=False)